In [4]:
!pip install requests
!pip install beautifulsoup4
!pip install wikipedia
!pip install PyDictionary
!pip install pytrends

  Preparing metadata (setup.py) ... done
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16109 sha256=5e6e0dc3b97d8e909ac47b59a976b295dd8ffb6d14a332d35efddd468c0d3bfc
  Stored in directory: /Users/grahamwaters/Library/Caches/pip/wheels/39/b3/27/fe5d0e140183e8001d8e09a9c290c5313490e3899f09b06a7f
Successfully built pytrends


In [23]:
import pytrends
from pytrends.request import TrendReq
import wikipedia
import requests
from bs4 import BeautifulSoup
import random
import PyDictionary
from PyDictionary import PyDictionary
#import spacy
import pandas as pd
import re

dir(pytrends)
pytrends = TrendReq(hl='en-US', tz=360) # create a pytrends object
#pytrends.trending_searches(pn='united_states') # trending searches in real time for United States
# create a dictionary of the top 1000 trending searches for this year in the United States (as of 2022)
trending_searches_dict = pytrends.top_charts(
    date='2022',    # specifies the time period as the last year
    geo='US',            # specifies the country as the United States
)
real_time_trending_searches_dict = pytrends.trending_searches(pn='united_states') # trending searches in real time for United States
related_topics_dict = pytrends.related_topics() # related topics for the top 10 trending searches in real time for United States


# show how many trending searches there are
print(f'Retrieved {len(trending_searches_dict)} trending searches.')
# show the top 10 trending searches in real time for United States
trending_searches_dict.head(10)

Retrieved 10 trending searches.


,title,exploreQuery
0,Wordle,
1,Election results,
2,Betty White,
3,Queen Elizabeth,
4,Bob Saget,
5,Ukraine,
6,Mega Millions,
7,Powerball numbers,
8,Anne Heche,
9,Jeffrey Dahmer,


In [24]:
related_topics_dict 

{}

In [25]:
real_time_trending_searches_dict

,0
0,Earthquake
1,The Specials
2,Weather Chicago
3,Boston Bruins
4,Green Bay Packers
5,New York Giants
6,Trump
7,Drew Griffin CNN
8,Harvey Weinstein
9,Jalen Hurts injury


In [2]:
def check_for_badwords():
  if any(re.search(r'\b' + word + r'\b', definition) for word in ['sex', 'porn','fuck','shit','damn','ass','cock','whore','nigger','slut','blowjob','faggot''boob', 'breast', 'cunt', 'pussy', 'dick', 'naked', 'nud*', 'nipple']):
    return True
  else:
    return False # no bad words

In [3]:
import requests
import random
import re
from bs4 import BeautifulSoup
from datetime import datetime
from PyDictionary import PyDictionary
import wikipedia

dictionary = PyDictionary() # instantiate 

def get_wiki_definition(card):
  # Get the summary of the Wikipedia page
  summary = wikipedia.summary(card)
  return summary

# # Example usage:
# card = "John Smith"
# definition = get_definition(card)
# print(definition)  # prints the summary of the Wikipedia page "John Smith"

def get_definition(card):
  # Check the type of the card
  if is_slang_phrase(card):
    # Get the definition of the slang phrase from Urban Dictionary
    url = "https://api.urbandictionary.com/v0/define?term=" + card
    response = requests.get(url)
    data = response.json()
    definition = data['list'][0]['definition']
  elif is_person(card):
    # Get the summary of the person from Wikipedia
    definition = get_wiki_definition(card[0]) # for a card title get the definition (summary) of the wikipedia page.
  else:
    # this is a word so use PyDictionary library.
    definition = dictionary.meaning(str(card[0])) # this could have mult. meanings returned. 
    definition = definition[0] # the first meaning. 

def get_random_slang_phrase():
  # Make a request to the Urban Dictionary API to get a random slang phrase
  url = "https://api.urbandictionary.com/v0/random"
  response = requests.get(url)
  data = response.json()
  slang_phrase = data['list'][0]['word']
  return slang_phrase

def get_random_wiki_person():
  # Make a request to Wikipedia API to get a random person
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  person = data['query']['random'][0]['title']
  return person

def get_random_wiki_word():
  # Make a request to Wikipedia API to get a random word
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  word = data['query']['random'][0]['title']
  return word

def create_a_card_deck(num_cards=30):
  # create 30 random cards with equal distributions of slang, people, and words.
  # avoid profanity, or lude sexual topics.
  
  # Create a list of all the cards
  cards = []

  # Initialize a dictionary to keep track of the count of each type of card
  counts = {
    "slang": 0,
    "people": 0,
    "words": 0
  }

  # While there are fewer than 30 cards in the deck, generate a new card
  while len(cards) < num_cards:
    # Find the type of card with the lowest count
    min_count_type = min(counts, key=counts.get)

    # Generate a new card of that type
    if min_count_type == "slang":
      slang_phrase = get_random_slang_phrase()
      new_card = (slang_phrase, "slang", get_definition(slang_phrase))
    elif min_count_type == "people":
      person = get_random_wiki_person()
      new_card = (person, "people", get_definition(person))
    else:
      word = get_random_wiki_word()
      new_card = (word, "word", get_definition(word))

    # Add the new card to the list and update the count
    cards.append(new_card)
    counts[min_count_type] += 1 
  return cards

def is_slang_phrase(card):
  if card[1] == 'slang':
    return True
  else:
    return False

def is_person(card):
  if card[1] == 'person':
    return True
  else:
    return False

def filter_cards(cards):
  # create a function to filter out inappropriate content
  filtered_cards = []
  for card in cards:

    # check if the card contains profanity or sexual content
    print("checking card: {}".format(card))
    definition = card[2] # definition
    card_type = card[1] # i.e. 'slang'
    word = card[0] # word 
    try:
      if "profanity" in definition or "sex" in definition or "lewd" in definition:
        continue
      elif check_for_badwords():
        continue
      else:
        filtered_cards.append(card)
    except Exception as e:
      print(f'error with finding the meaning in the soup object. soup comes from the urban dictionary page.')
      print(e)
  return filtered_cards

# generate a deck of cards
card_deck = create_a_card_deck()

# filter the deck for inappropriate content
filtered_deck = filter_cards(card_deck)

# print the cards
print('Generated {} cards'.format(len(filtered_deck)))

KeyError: 0

In [ ]:
# print the filtered cards
for card in filtered_deck:
  print(card)


In [ ]:
# Example usage:
card = "dope"
definition = get_definition(card)
card_type = "phrase"
card_preview = 

# (slang_phrase, "slang", get_definition(slang_phrase))
    


print(definition)  # prints the definition of the slang phrase "dope"

card = "John Smith"
definition = get_definition(card)
print(definition)  # prints the summary of the person "John Smith"

card = "tree"
definition = get_definition(card)
print(definition)  # prints the definition of the word "tree"
